In [1]:
#import libraries
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

Get Url and make beautiful soup object

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup =  BeautifulSoup(source,'lxml')

Get table and DF column names and row data

In [3]:
tbl =soup.find('div',class_="mw-parser-output").table
col_names = list()
for x in tbl.find_all('th'):
    col_names.append(x.text)
col_names[2] = col_names[2].replace("\n",'')
rows = []
row = []
i = 0
for x in tbl.find_all('tr'):
    for z in x.find_all('td'):
        row.append(z.text)
    if (i==1):
        row[2] = row[2].replace("\n",'')
    rows.append(row)
    row = []
    i =1
rows = rows[1:]

Create DF.Drop rows in which borough is not assigned.

In [4]:
df = pd.DataFrame(rows, columns = col_names)
df.drop(df[df.Borough == "Not assigned"].index,inplace=True)

Create function to aggregate neighbourhood column

In [5]:
def neigh_add(arr):
    l = len(arr)
    s= ''
    index = 0
    for x in arr:
        if index == l-1:
            s= s+x
        else:
            s = s+ x + ', '
        index +=1
    return s

In [6]:
df = df.groupby(['Postcode','Borough']).Neighbourhood.agg(neigh_add).reset_index()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
df.Neighbourhood =  np.where(df.Neighbourhood == "Not assigned",df.Borough,df.Neighbourhood)

In [8]:
 df.shape

(103, 3)

In [25]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [32]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
data = pd.read_csv('Geospatial_Coordinates.csv')

In [33]:
data = data.rename(columns ={'Postal Code':'Postcode'})

In [34]:
data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
df = pd.merge(df,data)

In [38]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
